In [1]:
import datetime
import json
import pathlib
import re

import numpy as np
import rioxarray
import xarray as xr

In [2]:
fn = "resize"
size = "50%"
url = "https://eoepca.org/media_portal/images/logo6_med.original.png"

xcengine_config = dict(
    workflow_id="xceconvert-11",
    container_image_tag="quay.io/bcdev/xcetest-convert:11",
    environment_file="environment.yml",
)

In [3]:
da = rioxarray.open_rasterio(url)

/home/pont/mambaforge/envs/xcube2505a/lib/python3.13/site-packages/rioxarray/_io.py:1143: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  warnings.warn(str(rio_warning.message), type(rio_warning.message))  # type: ignore


In [4]:
da_small = da.coarsen(x=2, y=2).mean().astype(np.uint8)

In [5]:
ds = xr.Dataset(dict(v=da_small))

In [6]:
def write_stac_item(output_basename, suffix):
    subdir = pathlib.Path(suffix)
    subdir.mkdir()
    now = datetime.datetime.now(datetime.UTC)
    item_stac = {
        "type": "Feature",
        "stac_version": "1.1.0",
        "stac_extensions": [],
        "id": f"{output_basename}-{now.timestamp():.9f}",
        "geometry": {
            "type": "Polygon",
            "coordinates": [[[0, -90], [0, 90], [360, 90], [360, -90], [0, -90]]],
        },
        "bbox": [0, -90, 360, 90],
        "properties": {
            "start_datetime": "2000-01-01T00:00:00Z",
            "end_datetime": "2001-01-01T00:00:00Z",
            "datetime": None,
        },
        "links": [
            {"rel": "root", "href": "../catalog.json", "type": "application/json"},
            {"rel": "parent", "href": "../catalog.json", "type": "application/json"},
        ],
        "assets": {
            output_basename: {
                "href": f"./{output_basename}.{suffix}",
                "type": "image/png",
                "title": output_basename,
                "roles": ["data", "visual"],
            }
        },
    }
    
    item_stac["properties"] |= {
        "xcengine_config:" + k: v for k, v in xcengine_config.items()
    }
    
    with open(subdir / (output_basename + ".json"), "w") as fh:
        json.dump(item_stac, fh, indent=2)

In [7]:
output_basename = (
    re.sub("[.]png$", "", pathlib.Path(url).parts[-1]) + "-resize"
)
output_types = ["png"]
for output_type in output_types:
    write_stac_item(output_basename, output_type)
    
output_jsonname = output_basename + ".json"

In [8]:
da_small.rio.to_raster(pathlib.Path("png") / (output_basename + ".png"))
ds.to_zarr(pathlib.Path("zarr") / (output_basename + ".zarr"))

In [9]:
stac_item_links = [
    {
        "rel": "item",
        "href": f"./{output_type}/{output_jsonname}",
        "type": "application/geo+json",
    } for output_type in output_types
]

In [10]:
catalog_stac = {
    "type": "Catalog",
    "id": "catalog",
    "stac_version": "1.1.0",
    "description": "Root catalog",
    "links": [
        {"rel": "root", "href": "./catalog.json", "type": "application/json"},
    ] + stac_item_links,
}

with open("catalog.json", "w") as fh:
    json.dump(catalog_stac, fh, indent=2)